<a href="https://colab.research.google.com/github/AmitNikhade/AmitNikhade/blob/main/QandA_8.6.22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip uninstall pipelines -y

Found existing installation: pipelines 0.0.1
Uninstalling pipelines-0.0.1:
  Successfully uninstalled pipelines-0.0.1


In [ ]:
import zipfile

with zipfile.ZipFile("/content/a.zip") as zf:
    zf.extractall()

In [38]:
!pip uninstall transformers -y
!pip uninstall sentencepiece
!pip install -U transformers==3.0.0
!pip install -U sentencepiece==0.1.91
!python -m nltk.downloader punkt



Found existing installation: transformers 4.19.2
Uninstalling transformers-4.19.2:
  Successfully uninstalled transformers-4.19.2
Found existing installation: sentencepiece 0.1.91
Uninstalling sentencepiece-0.1.91:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/_sentencepiece.cpython-37m-x86_64-linux-gnu.so
    /usr/local/lib/python3.7/dist-packages/sentencepiece-0.1.91.dist-info/*
    /usr/local/lib/python3.7/dist-packages/sentencepiece.py
Proceed (y/n)? y
  Successfully uninstalled sentencepiece-0.1.91
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-3.0.0-py3-none-any.whl (754 kB)
  Using cached sentencepiece-0.1.96-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Using cached tokenizers-0.8.0rc4-cp37-cp37m-manylinux1_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfull

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached sentencepiece-0.1.91-cp37-cp37m-manylinux1_x86_64.whl (1.1 MB)
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.96
    Uninstalling sentencepiece-0.1.96:
      Successfully uninstalled sentencepiece-0.1.96


/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
import itertools
import logging
from typing import Optional, Dict, Union

from nltk import sent_tokenize

import torch
from transformers import(
    AutoModelForSeq2SeqLM, 
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
)

logger = logging.getLogger(__name__)

class QGPipeline:
    """Poor man's QG pipeline"""
    def __init__(
        self,
        model: PreTrainedModel,
        tokenizer: PreTrainedTokenizer,
        ans_model: PreTrainedModel,
        ans_tokenizer: PreTrainedTokenizer,
        qg_format: str,
        use_cuda: bool
    ):
        self.model = model
        self.tokenizer = tokenizer

        self.ans_model = ans_model
        self.ans_tokenizer = ans_tokenizer

        self.qg_format = qg_format

        self.device = "cuda" if torch.cuda.is_available() and use_cuda else "cpu"
        self.model.to(self.device)

        if self.ans_model is not self.model:
            self.ans_model.to(self.device)

        assert self.model.__class__.__name__ in ["T5ForConditionalGeneration", "BartForConditionalGeneration"]
        
        if "T5ForConditionalGeneration" in self.model.__class__.__name__:
            self.model_type = "t5"
        else:
            self.model_type = "bart"

    def __call__(self, inputs: str):
        inputs = " ".join(inputs.split())
        sents, answers = self._extract_answers(inputs)
        flat_answers = list(itertools.chain(*answers))
        
        if len(flat_answers) == 0:
          return []

        if self.qg_format == "prepend":
            qg_examples = self._prepare_inputs_for_qg_from_answers_prepend(inputs, answers)
        else:
            qg_examples = self._prepare_inputs_for_qg_from_answers_hl(sents, answers)
        
        qg_inputs = [example['source_text'] for example in qg_examples]
        questions = self._generate_questions(qg_inputs)
        output = [{'answer': example['answer'], 'question': que} for example, que in zip(qg_examples, questions)]
        return output
    
    def _generate_questions(self, inputs):
        inputs = self._tokenize(inputs, padding=True, truncation=True)
        
        outs = self.model.generate(
            input_ids=inputs['input_ids'].to(self.device), 
            attention_mask=inputs['attention_mask'].to(self.device), 
            max_length=32,
            num_beams=4,
        )
        
        questions = [self.tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]
        return questions
    
    def _extract_answers(self, context):
        sents, inputs = self._prepare_inputs_for_ans_extraction(context)
        inputs = self._tokenize(inputs, padding=True, truncation=True)

        outs = self.ans_model.generate(
            input_ids=inputs['input_ids'].to(self.device), 
            attention_mask=inputs['attention_mask'].to(self.device), 
            max_length=32,
        )
        
        dec = [self.ans_tokenizer.decode(ids, skip_special_tokens=False) for ids in outs]
        answers = [item.split('<sep>') for item in dec]
        answers = [i[:-1] for i in answers]
        
        return sents, answers
    
    def _tokenize(self,
        inputs,
        padding=True,
        truncation=True,
        add_special_tokens=True,
        max_length=512
    ):
        inputs = self.tokenizer.batch_encode_plus(
            inputs, 
            max_length=max_length,
            add_special_tokens=add_special_tokens,
            truncation=truncation,
            padding="max_length" if padding else False,
            pad_to_max_length=padding,
            return_tensors="pt"
        )
        return inputs
    
    def _prepare_inputs_for_ans_extraction(self, text):
        sents = sent_tokenize(text)

        inputs = []
        for i in range(len(sents)):
            source_text = "extract answers:"
            for j, sent in enumerate(sents):
                if i == j:
                    sent = "<hl> %s <hl>" % sent
                source_text = "%s %s" % (source_text, sent)
                source_text = source_text.strip()
            
            if self.model_type == "t5":
                source_text = source_text + " </s>"
            inputs.append(source_text)

        return sents, inputs
    
    def _prepare_inputs_for_qg_from_answers_hl(self, sents, answers):
        inputs = []
        for i, answer in enumerate(answers):
            if len(answer) == 0: continue
            for answer_text in answer:
                sent = sents[i]
                sents_copy = sents[:]
                
                answer_text = answer_text.strip()
                
                ans_start_idx = sent.index(answer_text)
                
                sent = f"{sent[:ans_start_idx]} <hl> {answer_text} <hl> {sent[ans_start_idx + len(answer_text): ]}"
                sents_copy[i] = sent
                
                source_text = " ".join(sents_copy)
                source_text = f"generate question: {source_text}" 
                if self.model_type == "t5":
                    source_text = source_text + " </s>"
                
                inputs.append({"answer": answer_text, "source_text": source_text})
        
        return inputs
    
    def _prepare_inputs_for_qg_from_answers_prepend(self, context, answers):
        flat_answers = list(itertools.chain(*answers))
        examples = []
        for answer in flat_answers:
            source_text = f"answer: {answer} context: {context}"
            if self.model_type == "t5":
                source_text = source_text + " </s>"
            
            examples.append({"answer": answer, "source_text": source_text})
        return examples

    
class MultiTaskQAQGPipeline(QGPipeline):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
    def __call__(self, inputs: Union[Dict, str]):
        if type(inputs) is str:
            # do qg
            return super().__call__(inputs)
        else:
            # do qa
            return self._extract_answer(inputs["question"], inputs["context"])
    
    def _prepare_inputs_for_qa(self, question, context):
        source_text = f"question: {question}  context: {context}"
        if self.model_type == "t5":
            source_text = source_text + " </s>"
        return  source_text
    
    def _extract_answer(self, question, context):
        source_text = self._prepare_inputs_for_qa(question, context)
        inputs = self._tokenize([source_text], padding=False)
    
        outs = self.model.generate(
            input_ids=inputs['input_ids'].to(self.device), 
            attention_mask=inputs['attention_mask'].to(self.device), 
            max_length=16,
        )

        answer = self.tokenizer.decode(outs[0], skip_special_tokens=True)
        return answer


class E2EQGPipeline:
    def __init__(
        self,
        model: PreTrainedModel,
        tokenizer: PreTrainedTokenizer,
        use_cuda: bool
    ) :

        self.model = model
        self.tokenizer = tokenizer

        self.device = "cuda" if torch.cuda.is_available() and use_cuda else "cpu"
        self.model.to(self.device)

        assert self.model.__class__.__name__ in ["T5ForConditionalGeneration", "BartForConditionalGeneration"]
        
        if "T5ForConditionalGeneration" in self.model.__class__.__name__:
            self.model_type = "t5"
        else:
            self.model_type = "bart"
        
        self.default_generate_kwargs = {
            "max_length": 256,
            "num_beams": 4,
            "length_penalty": 1.5,
            "no_repeat_ngram_size": 3,
            "early_stopping": True,
        }
    
    def __call__(self, context: str, **generate_kwargs):
        inputs = self._prepare_inputs_for_e2e_qg(context)

        # TODO: when overrding default_generate_kwargs all other arguments need to be passsed
        # find a better way to do this
        if not generate_kwargs:
            generate_kwargs = self.default_generate_kwargs
        
        input_length = inputs["input_ids"].shape[-1]
        
        # max_length = generate_kwargs.get("max_length", 256)
        # if input_length < max_length:
        #     logger.warning(
        #         "Your max_length is set to {}, but you input_length is only {}. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)".format(
        #             max_length, input_length
        #         )
        #     )

        outs = self.model.generate(
            input_ids=inputs['input_ids'].to(self.device), 
            attention_mask=inputs['attention_mask'].to(self.device),
            **generate_kwargs
        )

        prediction = self.tokenizer.decode(outs[0], skip_special_tokens=True)
        questions = prediction.split("<sep>")
        questions = [question.strip() for question in questions[:-1]]
        return questions
    
    def _prepare_inputs_for_e2e_qg(self, context):
        source_text = f"generate questions: {context}"
        if self.model_type == "t5":
            source_text = source_text + " </s>"
        
        inputs = self._tokenize([source_text], padding=False)
        return inputs
    
    def _tokenize(
        self,
        inputs,
        padding=True,
        truncation=True,
        add_special_tokens=True,
        max_length=512
    ):
        inputs = self.tokenizer.batch_encode_plus(
            inputs, 
            max_length=max_length,
            add_special_tokens=add_special_tokens,
            truncation=truncation,
            padding="max_length" if padding else False,
            pad_to_max_length=padding,
            return_tensors="pt"
        )
        return inputs


SUPPORTED_TASKS = {
    "question-generation": {
        "impl": QGPipeline,
        "default": {
            "model": "valhalla/t5-small-qg-hl",
            "ans_model": "valhalla/t5-small-qa-qg-hl",
        }
    },
    "multitask-qa-qg": {
        "impl": MultiTaskQAQGPipeline,
        "default": {
            "model": "valhalla/t5-small-qa-qg-hl",
        }
    },
    "e2e-qg": {
        "impl": E2EQGPipeline,
        "default": {
            "model": "valhalla/t5-small-e2e-qg",
        }
    }
}

def pipeline(
    task: str,
    model: Optional = None,
    tokenizer: Optional[Union[str, PreTrainedTokenizer]] = None,
    qg_format: Optional[str] = "highlight",
    ans_model: Optional = None,
    ans_tokenizer: Optional[Union[str, PreTrainedTokenizer]] = None,
    use_cuda: Optional[bool] = True,
    **kwargs,
):
    # Retrieve the task
    if task not in SUPPORTED_TASKS:
        raise KeyError("Unknown task {}, available tasks are {}".format(task, list(SUPPORTED_TASKS.keys())))

    targeted_task = SUPPORTED_TASKS[task]
    task_class = targeted_task["impl"]

    # Use default model/config/tokenizer for the task if no model is provided
    if model is None:
        model = targeted_task["default"]["model"]
    
    # Try to infer tokenizer from model or config name (if provided as str)
    if tokenizer is None:
        if isinstance(model, str):
            tokenizer = model
        else:
            # Impossible to guest what is the right tokenizer here
            raise Exception(
                "Impossible to guess which tokenizer to use. "
                "Please provided a PretrainedTokenizer class or a path/identifier to a pretrained tokenizer."
            )
    
    # Instantiate tokenizer if needed
    if isinstance(tokenizer, (str, tuple)):
        if isinstance(tokenizer, tuple):
            # For tuple we have (tokenizer name, {kwargs})
            tokenizer = AutoTokenizer.from_pretrained(tokenizer[0], **tokenizer[1])
        else:
            tokenizer = AutoTokenizer.from_pretrained(tokenizer)
    
    # Instantiate model if needed
    if isinstance(model, str):
        model = AutoModelForSeq2SeqLM.from_pretrained(model)
    
    if task == "question-generation":
        if ans_model is None:
            # load default ans model
            ans_model = targeted_task["default"]["ans_model"]
            ans_tokenizer = AutoTokenizer.from_pretrained(ans_model)
            ans_model = AutoModelForSeq2SeqLM.from_pretrained(ans_model)
        else:
            # Try to infer tokenizer from model or config name (if provided as str)
            if ans_tokenizer is None:
                if isinstance(ans_model, str):
                    ans_tokenizer = ans_model
                else:
                    # Impossible to guest what is the right tokenizer here
                    raise Exception(
                        "Impossible to guess which tokenizer to use. "
                        "Please provided a PretrainedTokenizer class or a path/identifier to a pretrained tokenizer."
                    )
            
            # Instantiate tokenizer if needed
            if isinstance(ans_tokenizer, (str, tuple)):
                if isinstance(ans_tokenizer, tuple):
                    # For tuple we have (tokenizer name, {kwargs})
                    ans_tokenizer = AutoTokenizer.from_pretrained(ans_tokenizer[0], **ans_tokenizer[1])
                else:
                    ans_tokenizer = AutoTokenizer.from_pretrained(ans_tokenizer)

            if isinstance(ans_model, str):
                ans_model = AutoModelForSeq2SeqLM.from_pretrained(ans_model)
    
    if task == "e2e-qg":
        return task_class(model=model, tokenizer=tokenizer, use_cuda=use_cuda)
    elif task == "question-generation":
        return task_class(model=model, tokenizer=tokenizer, ans_model=ans_model, ans_tokenizer=ans_tokenizer, qg_format=qg_format, use_cuda=use_cuda)
    else:
        return task_class(model=model, tokenizer=tokenizer, ans_model=model, ans_tokenizer=tokenizer, qg_format=qg_format, use_cuda=use_cuda)

In [12]:
nlp =pipeline("question-generation", model="valhalla/t5-base-qg-hl")

In [13]:
# f = open("/content/sh.txt")
# f = f.read().strip().replace("\n","").replace("  ","")
# text= f
text = """Donning many hats of a speaker, author, entrepreneur and navigator, Yogesh is among Asia’s top leadership and business strategy authorities whose work has impacted entrepreneurs and enterprises across the globe. With a master’s degree in Transactional Analysis, Yogesh is a change catalyst who consults internationally to large companies, small and medium enterprises, and government organizations on his unique model of people and business growth & transformation, business model re-innovation, and impactful leadership development.
Yogesh began his career as an entrepreneur at the early age of 16, at his father’s shop after which, he ran an adventure campsite, and then, a real estate company. He took every success as a blessing and every failure as a learning.
Yogesh is an avid reader and his keen interest in Psychology helped him understand business patterns and business behaviors of people. He learned his basic lessons of Psychology based on the Freudian theory of Transactional Analysis. Based on his observations of people behaviors, he has created effective models like the “Leadership Imbalance Model ®” and the “CPC Model of Business Scalability ®.” His ability to understand the scientific reasons behind people’s behavioral patterns helped him to gain the confidence of all stakeholders and handle complex situations in businesses where he advised, consulted and trained.
Yogesh’s clear, data-driven methodologies have helped companies across the globe design and implement growth strategies and drive real results, both in terms of building capability and significant financial returns.
Consistent growth is one of his greatest motivators, and being equally humble and assertive has attracted a lot of leaders to learn from him. He has dedicated his life to helping leaders seek clarity and build tolerance through his signature concept of “What great leaders do.” He focuses on developing and correcting behaviors and habits in leaders in order to create the impact required at the marketplace. His “Leadership Imbalance Model ®” helps leaders realize “What they are doing but shouldn’t do.”
His education, observations of the modern world, corporate experience of solving problems, innate curiosity and eagerness to assist people to attain their true potential has led him to become a dependable leader, entrepreneur and a trusted advisor to his clients worldwide.Yogesh began his career as an entrepreneur at the early age of 16, at his father’s shop after which, he ran an adventure campsite, and then, a real estate company. He took every success as a blessing and every failure as a learning.
Yogesh is an avid reader and his keen interest in Psychology helped him understand business patterns and business behaviors of people. He learned his basic lessons of Psychology based on the Freudian theory of Transactional Analysis. Based on his observations of people behaviors, he has created effective models like the “Leadership Imbalance Model ®” and the “CPC Model of Business Scalability ®.” His ability to understand the scientific reasons behind people’s behavioral patterns helped him to gain the confidence of all stakeholders and handle complex situations in businesses where he advised, consulted and trained.
Yogesh’s clear, data-driven methodologies have helped companies across the globe design and implement growth strategies and drive real results, both in terms of building capability and significant financial returns.
Consistent growth is one of his greatest motivators, and being equally humble and assertive has attracted a lot of leaders to learn from him. He has dedicated his life to helping leaders seek clarity and build tolerance through his signature concept of “What great leaders do.” He focuses on developing and correcting behaviors and habits in leaders in order to create the impact required at the marketplace. His “Leadership Imbalance Model ®” helps leaders realize “What they are doing but shouldn’t do.”His education, observations of the modern world, corporate experience of solving problems, innate curiosity and eagerness to assist people to attain their true potential has led him to become a dependable leader, entrepreneur and a trusted advisor to his clients worldwide.

"""

In [4]:
# print(nlp("Day one: finding your breath.“Take a moment to listen to your breathing,” the recording of Dr. McCourt said, through the car’s speakers. “Don’t try to control it, just feel the air as it enters and exits your nose.”Garry slammed his brakes and his horn. Just like the driver ahead. Just like the driver behind.“Stop fucking honking at me asswipe!” he screamed over his shoulder. He could see the driver behind him, screaming himself red. Look at this guy! What an idiot.Somewhere a light turned green."))

In [14]:

from tqdm.notebook import tqdm_notebook
import uuid
data = []
text_ = text.split('.')
tm = 0
for n,s in enumerate(tqdm_notebook(text_)):
  # print(nlp(s))
  # if n == 2000:
  #   break
  # else:
    # print(n)
  try:
    qa = nlp(s)
    print(qa)
    for i in qa:
      if str(i['answer']) in str(s):
        # print([str(uuid.uuid4()).replace("-",""),s,i['question'],i['answer'],str(text.index(i['answer']))])
        data.append([str(uuid.uuid4()).replace("-",""),s,i['question'],i['answer'],str(text.index(i['answer']))])
          # tm = tm+1
      
          
  except:
    pass 

  
     
  # # break

  0%|          | 0/27 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1500: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


[{'answer': 'Yogesh', 'question': "Who is one of Asia's top leadership and business strategy authorities?"}]
[{'answer': 'Transactional Analysis', 'question': "What is Yogesh's master's degree in?"}]
[{'answer': 'entrepreneur', 'question': 'What occupation did Yogesh begin his career as?'}, {'answer': '16', 'question': 'At what age did Yogesh start his career as an entrepreneur?'}]
[{'answer': 'blessing', 'question': 'What did he take every success as?'}]
[{'answer': 'Freudian theory of Transactional Analysis', 'question': 'What theory did he use to learn basic lessons of Psychology?'}, {'answer': 'Freudian theory of Transactional Analysis', 'question': 'What theory did he use to learn basic lessons of Psychology?'}]
[{'answer': 'Leadership Imbalance Model ®', 'question': 'What is the name of the model that he has created?'}]
[{'answer': 'understand the scientific reasons behind people’s behavioral patterns', 'question': 'How did he gain the confidence of all stakeholders?'}]
[{'answer

In [1]:
# data

In [7]:
import pandas as pd
 
# intialise data of lists.
data = {
    # 'id':[i[0] for i in data],
    #     'context':[i[1] for i in data],
        'question':[i[2] for i in data],
        'answer_text':[i[3] for i in data],
        # 'answer_start':[i[4] for i in data],
        # 'answers':[{'answer_start':[int(i[4])],'text':[i[3]]} for i in data]
    
        }
 
# Create DataFrame
train = pd.DataFrame(data)
 
# Print the output.
# print(df)

In [8]:
train

,question,answer_text
0,Who is one of Asia's top leadership and busine...,Yogesh
1,What is Yogesh's master's degree in?,Transactional Analysis
2,What occupation did Yogesh begin his career as?,entrepreneur
3,At what age did Yogesh start his career as an ...,16
4,What did he take every success as?,blessing
5,What theory did he use to learn basic lessons ...,Freudian theory of Transactional Analysis
6,What theory did he use to learn basic lessons ...,Freudian theory of Transactional Analysis
7,What is the name of the model that he has crea...,Leadership Imbalance Model ®
8,How did he gain the confidence of all stakehol...,understand the scientific reasons behind peopl...
9,Yogesh’s data-driven methodologies have helped...,building capability and significant financial ...


In [9]:
train.to_csv('file_dr_yo.csv')

In [1]:
# !pip uninstall transformers
# !pip install -U transformers==4.19.2

from transformers import AutoModelWithLMHead, AutoTokenizer
from  transformers  import  AutoTokenizer, AutoModelWithLMHead, pipeline
data1 = []
model_name = "MaRiOrOsSi/t5-base-finetuned-question-answering"
tokenizer1 = AutoTokenizer.from_pretrained(model_name)
model1 = AutoModelWithLMHead.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

def get_question(answer, context, max_length=64):
  input_text = "answer: %s  context: %s </s>" % (answer, context)
  features = tokenizer([input_text], return_tensors='pt')

  output = model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)

  return tokenizer.decode(output[0], skip_special_tokens=True,clean_up_tokenization_spaces=True)

context = "Yogesh is among Asia’s top leadership and business strategy authorities whose work has impacted entrepreneurs and enterprises across the globe."
for i in context.split(" "):
  # print(get_question(i, context)[10:])
  input = f"question: {(get_question(i, context)[10:])} context: {context}"
  encoded_input = tokenizer1([input],
                              return_tensors='pt',
                              max_length=512,
                              truncation=True)
  output = model1.generate(input_ids = encoded_input.input_ids,
                              attention_mask = encoded_input.attention_mask)
  output = tokenizer1.decode(output[0], skip_special_tokens=True)
  print((get_question(i, context)[10:]),output)
  data1.append([(get_question(i, context)[10:]),output])

# answer = "Asia’s top leadership and business strategy authorities"

# get_question(answer, context)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Who is one of Asia's top leadership and business strategy authorities? Yogesh
What is Yogesh's profession? leadership and business strategy authorities
Where does Yogesh rank among Asia's top leadership and business strategy authorities? 
Yogesh is one of which countries top leadership and business strategy authorities? Asia
What is Yogesh's rank in Asia? Top leadership and business strategy authorities
What is Yogesh's profession? leadership and business strategy authorities
What is Yogesh's professional background? leadership and business strategy
What type of strategy has Yogesh developed? leadership and business strategy
What is Yogesh's specialty? leadership and business strategy
What is Yogesh a part of? Asia’s top leadership and business strategy authorities
Who is Yogesh? Asia’s top leadership and business strategy authorities
What has Yogesh done that has impacted entrepreneurs and enterprises across the globe? Yogesh has led and business strategy authorities.
How has Yogesh's

In [26]:
import pandas as pd

data = {
 
        'question':[i[0] for i in data1],
        'answer_text':[i[1] for i in data1],
   
        }
 
# Create DataFrame
train1 = pd.DataFrame(data)
 

In [28]:
final_data = pd.concat([train, train1], axis=0)

NameError: ignored